In [1]:
from matplotlib import pyplot as plt
import numpy as np
from eeg_v2 import EEG
from utils import chunks, baseline_calc, butter_bandpass_filter
import random
from mne.preprocessing import ICA

In [2]:
# Reading raw data
path_edf="./edf/1578_alice/edf/A0001578.edf"
path_stage="./edf/1578_alice/csv/STAGE.csv"

eeg = EEG(path_edf=path_edf, path_stage=path_stage)

/home/tudo/Desktop/code/ica_paper/eeg_v2.py:17: RuntimeWarning: Channel names are not unique, found duplicates for: {'EMG Chin'}. Applying running numbers for duplicates.
  raw = read_raw_edf(self.path_edf, preload=True, verbose=0)


In [3]:
# Params
_window = 15
_fs = 500

In [4]:
# Preprocessing function
def f(eeg):
    
    # Getting randome chunk of data from raw signal
    _chunk = random.choice(list(chunks(random.choice(eeg.tasks), _window)))
    _min, _max = np.min(_chunk), np.max(_chunk)

    
    # Preprocessing
    # Getting sample data
    sample_raw = eeg.raw.copy().crop(_min, _max, include_tmax=False)
    eog_channels = eeg.eog_channels.copy().crop(_min, _max, include_tmax=False)

    # Apply baseline correction
    sample_raw_baseline = sample_raw.copy()
    sample_raw_baseline.apply_function(baseline_calc)
    # eog_channels.apply_function(baseline_calc)

    # Apply bandpass filter
    sample_raw_bandpass = sample_raw_baseline.copy()
    sample_raw_bandpass.apply_function(butter_bandpass_filter)
    
    # Applying ICA
    sample_raw_train = sample_raw_bandpass.copy()
    sample_raw_corrected = sample_raw_bandpass.copy()

    # Train
    ica = ICA(method="extended-infomax", random_state=1)
    ica.fit(sample_raw_corrected)
    
    return _min, _max, sample_raw, sample_raw_baseline, sample_raw_bandpass, sample_raw_corrected, ica

In [5]:
_min, _max, sample_raw, sample_raw_baseline, sample_raw_bandpass, sample_raw_corrected, ica = f(eeg)

Fitting ICA to data using 10 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 10
Computing Extended Infomax ICA


<ipython-input-4-36984df7469a>:28: DeprecationWarning: method='extended-infomax' is deprecated and will be removed in 0.19. If you want to use Extended Infomax, specify method='infomax' together with fit_params=dict(extended=True).
  ica = ICA(method="extended-infomax", random_state=1)


Fitting ICA took 3.8s.


In [ ]:
def plot(sig):
    n, l = s